# 时间序列预测批发价

In [7]:
import pandas as pd

# 读取数据
data = pd.read_excel('Q3-1.xlsx')

# 将销售日期列转换为日期数据类型
data['销售日期'] = pd.to_datetime(data['销售日期'])

# 按单品名称拆分数据
unique_products = data['单品名称'].unique()
product_forecasts = []

for product_name in unique_products:
    # 提取单品销售数据
    product_df = data[data['单品名称'] == product_name][['销售日期', '批发价格(元/千克)']]
    product_df.set_index('销售日期', inplace=True)
    
    # 获取最后一天的销量
    last_day_sales = product_df.iloc[-1]['批发价格(元/千克)']
    
    # 创建包含预测结果的DataFrame
    forecast_df = pd.DataFrame({
        '销售日期': ['2023-07-01'],
        '单品名称': [product_name],
        '批发价格(元/千克)': [last_day_sales]  # 将最后一天的销量作为预测结果
    })
    
    # 将预测结果添加到列表
    product_forecasts.append(forecast_df)

# 将所有预测结果合并成一个新的DataFrame
final_forecast_df = pd.concat(product_forecasts)

# 保存新的DataFrame到Excel文件
final_forecast_df.to_excel('product_forecasts.xlsx', index=False)


# 时间序列预测销量

In [95]:
import pandas as pd

# 读取数据
data = pd.read_excel('Q3-1.xlsx')

# 将销售日期列转换为日期数据类型
data['销售日期'] = pd.to_datetime(data['销售日期'])

# 按单品名称拆分数据
unique_products = data['单品名称'].unique()
product_forecasts = []

for product_name in unique_products:
    # 提取单品销售数据
    product_df = data[data['单品名称'] == product_name][['销售日期', '销量(千克)']]
    product_df.set_index('销售日期', inplace=True)
    
    # 获取最后一天的销量
    last_day_sales = product_df.iloc[-1]['销量(千克)']
    
    # 创建包含预测结果的DataFrame
    forecast_df = pd.DataFrame({
        '销售日期': ['2023-07-01'],
        '单品名称': [product_name],
        '销量(千克)': [last_day_sales]  # 将最后一天的销量作为预测结果
    })
    
    # 将预测结果添加到列表
    product_forecasts.append(forecast_df)

# 将所有预测结果合并成一个新的DataFrame
final_forecast_df = pd.concat(product_forecasts)

# 保存新的DataFrame到Excel文件
final_forecast_df.to_excel('product_sales.xlsx', index=False)

# 得到单品售价与销售量的关系

In [96]:
import pandas as pd

# 读取数据
data = pd.read_excel('Q3-1.xlsx')

# 定义阈值
threshold = 30

# 计算每个单品名称的行数
product_counts = data['单品名称'].value_counts()

# 获取满足条件的单品名称列表
valid_products = product_counts[product_counts >= threshold].index.tolist()

# 根据条件筛选数据
filtered_data = data[data['单品名称'].isin(valid_products)]

# 打印筛选后的数据
filtered_data.to_excel('Q3-2.xlsx', index=False)

In [97]:
# 读取数据
data = pd.read_excel('Q3-2.xlsx')

# 计算每个品类的销量的异常值阈值（可以根据具体需求选择不同的方法，如Z-score或IQR）
# 这里以Z-score为例
def remove_outliers(group):
    group['Z_score'] = (group['销量(千克)'] - group['销量(千克)'].mean()) / group['销量(千克)'].std()
    return group[group['Z_score'].abs() <= 3]  # 3是一个常用的阈值，你可以根据需要调整

filtered_data = data.groupby('单品名称').apply(remove_outliers)
filtered_data
filtered_data.to_excel('Q3-2.xlsx', index=False)


In [101]:
import pandas as pd
from pulp import LpVariable, LpProblem, LpMinimize, LpStatus, value
from sklearn.metrics import r2_score

# 从Excel文件读取数据
data = pd.read_excel('Q3-2.xlsx')
groups = data.groupby("单品名称")

results = []  # Initialize an empty list to store results

def getRsquared(A, B, C, price, wholesale_price, quantity):
    # 获取线性规划模型的结果
    A_value = value(A)
    B_value = value(B)
    C_value = value(C)

    # 使用线性规划模型的结果进行预测
    predicted_price = [A_value * q + B_value * w + C_value for q, w in zip(quantity, wholesale_price)]

    # 计算R方
    r_squared = r2_score(price, predicted_price)

    # 返回R方值
    return r_squared

def getLp(quantity, wholesale_price, price, name="NAME"):
    # 创建线性规划问题
    problem = LpProblem("Constrained Linear Regression", LpMinimize)

    # 创建变量
    A = LpVariable('A')
    B = LpVariable('B')
    C = LpVariable('C')
    E = [LpVariable(f'E{i}') for i in range(len(price))]

    # 创建约束条件
    constraint1 = A <= -0.0001
    constraint2 = B >= 1.000001

    # 创建目标函数
    error = sum(E[i] for i in range(len(price)))

    # 添加约束条件和目标函数到问题中
    problem += error
    problem += constraint1
    problem += constraint2

    # 添加误差约束条件
    for q, w, p, e in zip(quantity, wholesale_price, price, E):
        problem += e >= p - (A * q + B * w + C)
        problem += e >= (A * q + B * w + C) - p

    # 求解问题
    problem.solve()

    # 打印结果
    print(name)
    print("Status:", LpStatus[problem.status])
    print("A =", value(A))
    print("B =", value(B))
    print("C =", value(C))
    
    r_squared = getRsquared(A, B, C, price, wholesale_price, quantity)
    print("R-squared:", r_squared)
    
    # Append the results to the list
    results.append({'Name': name, 'A': value(A), 'B': value(B), 'C': value(C), 'R-squared': r_squared})

# Loop through groups and get results
for name, group in groups:
    quantity = group['销量(千克)'].tolist()
    wholesale_price = group['批发价格(元/千克)'].tolist()
    price = group['销售单价(元/千克)'].tolist()
    getLp(quantity, wholesale_price, price, name)

# Convert results list to a DataFrame
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_excel('linear_regression_results.xlsx', index=False)

results_df


/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  wa

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/l_/t15z68h5545gnmg_nk6tn8r00000gn/T/b2f9f82d8e0f40249a2af5f56a2335b1-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/l_/t15z68h5545gnmg_nk6tn8r00000gn/T/b2f9f82d8e0f40249a2af5f56a2335b1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 177 COLUMNS
At line 945 RHS
At line 1118 BOUNDS
At line 1207 ENDATA
Problem MODEL has 172 rows, 88 columns and 682 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 170 (-2) rows, 88 (0) columns and 680 (-2) elements
Perturbing problem by 0.001% of 5.3565212 - largest nonzero change 0 ( 0%) - largest zero change 5.5691007e-05
0  Obj 0.00063780681 Primal inf 413.10346 (85) Dual inf 2.7381342 (85) w.o. free dual inf (0)
78  Obj -2.0379116e+10 Primal inf 2.9117974e+

/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  wa

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/l_/t15z68h5545gnmg_nk6tn8r00000gn/T/5b33e4dc9fb24ae4b65736b43da43442-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/l_/t15z68h5545gnmg_nk6tn8r00000gn/T/5b33e4dc9fb24ae4b65736b43da43442-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 185 COLUMNS
At line 989 RHS
At line 1170 BOUNDS
At line 1263 ENDATA
Problem MODEL has 180 rows, 92 columns and 714 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 178 (-2) rows, 92 (0) columns and 712 (-2) elements
Perturbing problem by 0.001% of 6.3245553 - largest nonzero change 0 ( 0%) - largest zero change 5.6202706e-05
0  Obj 7.0204066e-05 Primal inf 55.397027 (89) Dual inf 4.0838378 (89) w.o. free dual inf (0)
78  Obj -1.2888499e+10 Primal inf 4.0463104e+

/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  wa

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/l_/t15z68h5545gnmg_nk6tn8r00000gn/T/78a62cceaa564b659ecf2cd6b22b5b2f-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/l_/t15z68h5545gnmg_nk6tn8r00000gn/T/78a62cceaa564b659ecf2cd6b22b5b2f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 135 COLUMNS
At line 714 RHS
At line 845 BOUNDS
At line 913 ENDATA
Problem MODEL has 130 rows, 67 columns and 514 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 128 (-2) rows, 67 (0) columns and 512 (-2) elements
Perturbing problem by 0.001% of 65.021533 - largest nonzero change 0 ( 0%) - largest zero change 8.7229678e-05
0  Obj 2.0125963e-05 Primal inf 31.91869 (64) Dual inf 22.6437 (64) w.o. free dual inf (0)
77  Obj 46.14785 Primal inf 10.051035 (25)
102  Ob

/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/opt/homebrew/anaconda3/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  wa

,Name,A,B,C,R-squared
0,七彩椒(2),-0.923220,1.000001,16.493273,-0.255171
1,上海青,-0.023293,3.427641,-6.687988,0.637010
2,云南油麦菜(份),-0.001111,1.000001,1.971109,-0.098279
3,云南生菜(份),-0.000100,1.000001,1.204997,0.263532
4,净藕(1),-0.000100,1.341363,1.245767,0.960147
5,双孢菇(盒),-0.002679,1.607143,0.083929,-0.080404
6,圆茄子(2),-0.000100,1.000001,3.590051,0.098435
7,奶白菜,-0.020629,1.000001,1.951188,-1.360226
8,姜蒜小米椒组合装(小份),-0.000100,1.000001,2.170697,-0.595989
9,娃娃菜,-0.000100,1.350540,0.490895,0.470174


# 合并批发价和多元线性回归结果

In [129]:
p=pd.read_excel('预测批发价.xlsx')
df = pd.read_excel('linear_regression_results.xlsx')
df['p']=p['批发价格(元/千克)']
df

,Name,A,B,C,R-squared,p
0,七彩椒(2),-0.923220,1.000001,16.493273,-0.255171,12.13
1,上海青,-0.023293,3.427641,-6.687988,0.637010,4.09
2,云南油麦菜(份),-0.001111,1.000001,1.971109,-0.098279,2.66
3,云南生菜(份),-0.000100,1.000001,1.204997,0.263532,2.84
4,净藕(1),-0.000100,1.341363,1.245767,0.960147,5.68
5,双孢菇(盒),-0.002679,1.607143,0.083929,-0.080404,3.49
6,圆茄子(2),-0.000100,1.000001,3.590051,0.098435,10.38
7,奶白菜,-0.020629,1.000001,1.951188,-1.360226,3.41
8,姜蒜小米椒组合装(小份),-0.000100,1.000001,2.170697,-0.595989,3.20
9,娃娃菜,-0.000100,1.350540,0.490895,0.470174,9.19


$$w=ax^2+(b-1)px+cx\space\space\space\space\space\space\space\space(满足a<0且b>1)$$ 
显然当且仅当$x=\frac{1-b}{2a}p$有w最大值
$Y=aX1+bX2+c$

In [130]:
df['销量']=(1-df['B'])/(2*df['A'])*df['p']
df['最大利润']=(df['A']*df['销量']*df['销量'])+(df['B']*df['p']*df['销量'])+df['C']*df['销量']
df['售价']=df['A']*df['销量']+df['B']*df['p']+df['C']
df['p'].rename('批发价')
df

,Name,A,B,C,R-squared,p,销量,最大利润,售价
0,七彩椒(2),-0.923220,1.000001,16.493273,-0.255171,12.13,0.000007,0.000188,28.623279
1,上海青,-0.023293,3.427641,-6.687988,0.637010,4.09,213.135987,504.394425,2.366538
2,云南油麦菜(份),-0.001111,1.000001,1.971109,-0.098279,2.66,0.001197,0.005543,4.631110
3,云南生菜(份),-0.000100,1.000001,1.204997,0.263532,2.84,0.014200,0.057439,4.044999
4,净藕(1),-0.000100,1.341363,1.245767,0.960147,5.68,9694.723400,76542.158670,7.895239
5,双孢菇(盒),-0.002679,1.607143,0.083929,-0.080404,3.49,395.533365,1832.661500,4.633393
6,圆茄子(2),-0.000100,1.000001,3.590051,0.098435,10.38,0.051900,0.725046,13.970056
7,奶白菜,-0.020629,1.000001,1.951188,-1.360226,3.41,0.000083,0.000443,5.361190
8,姜蒜小米椒组合装(小份),-0.000100,1.000001,2.170697,-0.595989,3.20,0.016000,0.085931,5.370699
9,娃娃菜,-0.000100,1.350540,0.490895,0.470174,9.19,16107.335975,181878.048921,11.291628


In [131]:
Y=pd.read_excel('预测销量.xlsx')
df['销量']=Y['销量(千克)_预测']
df['最大利润']=df['销量']*(df['售价']-df['p'])
df.sort_values(by='最大利润', ascending=False)  
df.to_excel('Q3-answer.xlsx', index=False)

In [5]:
import pandas as pd
df=pd.read_excel('Q3-answer.xlsx')
df=df.head(27)
df.to_excel('Q3-answer(final).xlsx', index=False)
df

,Name,A,B,C,R-squared,批发价,销量,最大利润,售价
0,洪湖藕带,-0.500221,1.000001,9.882105,0.828147,2.70,8.044053,79.492189,12.582106
1,圆茄子(2),-0.000100,1.000001,3.590051,0.098435,10.38,15.487791,55.602038,13.970056
2,云南油麦菜(份),-0.001111,1.000001,1.971109,-0.098279,2.66,22.482164,44.314818,4.631110
3,红椒(2),-0.425703,1.000001,6.576453,-0.002123,1.95,6.341162,41.702364,8.526454
4,净藕(1),-0.000100,1.341363,1.245767,0.960147,5.68,16.798942,37.213672,7.895239
5,小米椒(份),-0.012202,1.127713,2.654073,0.274876,2.33,11.772556,32.996809,5.132858
6,姜蒜小米椒组合装(小份),-0.000100,1.000001,2.170697,-0.595989,3.20,13.781611,29.915729,5.370699
7,奶白菜,-0.020629,1.000001,1.951188,-1.360226,3.41,14.176884,27.661788,5.361190
8,木耳菜,-0.002542,1.143508,2.290639,0.681384,2.10,11.104873,27.110572,4.541322
9,娃娃菜,-0.000100,1.350540,0.490895,0.470174,9.19,12.325243,25.903077,11.291628
